In [47]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os.path
from itertools import product
import gc


pd.options.display.float_format = '{0:.4f}'.format
sns.set(rc={'figure.figsize':(20,20)})
ran_seed = 1

In [49]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [51]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''  
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [53]:
PWD = r"C:\Users\Desktop\Sales"

train_file = os.path.join(PWD, "sales_train.csv")
test_file = os.path.join(PWD, "test.csv")

items_file = os.path.join(PWD, "items.csv")
item_categories_file = os.path.join(PWD, "item_categories.csv")
shops_file = os.path.join(PWD, "shops.csv")

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

items_df = pd.read_csv(items_file)
item_categories_df = pd.read_csv(item_categories_file)
shops_df = pd.read_csv(shops_file)

FileNotFoundError: File b'C:\\Users\\Desktop\\Sales\\sales_train.csv' does not exist

In [ ]:
train_df['date_block_num'].unique()

In [ ]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

In [ ]:
def create_shop_item_grid(data_df, index_cols):

    # For every month we create a grid from all shops/items combinations from that month
    grid = [] 
  
    for block_num in data_df['date_block_num'].unique():
        cur_shops = data_df.loc[data_df['date_block_num'] == block_num, 'shop_id'].unique()
        cur_items = data_df.loc[data_df['date_block_num'] == block_num, 'item_id'].unique()
        row_array = np.array(list(product(*[cur_shops, cur_items, [block_num]])), dtype='int32')
       
        grid.append(row_array)
    
    grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
        
    return grid

In [ ]:
def convert_monthly_sales(sales, feature_grid, index_cols):
        
    shop_item_mth_gb = sales.groupby(index_cols, as_index=False).agg(
        {"item_cnt_day" : "sum"}).rename(
        columns={"item_cnt_day" : "target"})
        
    shop_mth_gb = sales.groupby(["shop_id", "date_block_num"], as_index=False).agg(
        {"item_cnt_day" : "sum"}).rename(
        columns={"item_cnt_day" : "target_shop"})
    
    shop_item_gb = sales.groupby(["item_id", "date_block_num"], as_index=False).agg(
        {"item_cnt_day" : "sum"}).rename(
        columns={"item_cnt_day" : "target_item"})
    
    all_data = pd.merge(feature_grid, shop_item_mth_gb, how='left', on=index_cols).fillna(0)
    all_data = pd.merge(all_data, shop_mth_gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)
    all_data = pd.merge(all_data, shop_item_gb, how='left', on=['item_id', 'date_block_num']).fillna(0)
    
    
    return all_data

In [ ]:
def add_lag_features(time_data, lags, cols, index_cols):
    
    for month_shift in lags:
        shift_X = time_data[index_cols + cols].copy()
        shift_X['date_block_num'] = shift_X['date_block_num'] + month_shift
        suffix = "_lag-{:02d}".format(month_shift)
        for c in cols:
            shift_X.rename({c:"{}{}".format(c, suffix)}, 
                           axis=1, inplace=True)
        time_data = pd.merge(time_data, shift_X, how='left', on=index_cols).fillna(0)
        
    del shift_X
    
    return time_data
        

In [ ]:
def append_test_to_train(train_df, test_df):
    test_to_grid = test_df.copy().drop(['ID'], axis=1)
    
    train_cols = train_df.columns
    test_cols = test_to_grid.columns
    
    cols_to_add = [x for x in train_cols if x not in test_cols]
        
    df_to_add = pd.DataFrame(np.zeros((test_df.shape[0], len(cols_to_add)), np.int32),
                             columns=cols_to_add)
    
    test_to_grid = pd.concat([test_to_grid, df_to_add], axis=1, )
       
    test_to_grid['date_block_num'] = 34
    all_data = train_df.append(test_to_grid, sort=False)
    
    return all_data
    

In [ ]:
def append_test_to_monthly_sales(monthly_sales_df, test_df):
    
    test_to_append = test_df.copy().drop(['ID'], axis=1)
    
    mth_cols = monthly_sales_df.columns
    test_cols = test_to_append.columns
    
    cols_to_add = [x for x in mth_cols if x not in test_cols]
        
    df_to_add = pd.DataFrame(np.zeros((test_df.shape[0], len(cols_to_add)), np.int32),
                             columns=cols_to_add)
    
    test_to_append = pd.concat([test_to_append, df_to_add], axis=1)
       
    test_to_append['date_block_num'] = 34
    df = monthly_sales_df.append(test_to_append, sort=False)
    
    return df

In [ ]:
def gen_feature_matrix(train_df, test_df, items_df, lags, index_cols):

    grid = create_shop_item_grid(train_df, index_cols)

    all_data = convert_monthly_sales(train_df, grid, index_cols)
    all_data = append_test_to_monthly_sales(all_data, test_df)
    
    all_data = add_lag_features(all_data, lags, ['target', 'target_shop', 'target_item'], index_cols)
    
    # Category for each item
    item_category_mapping = items_df[['item_id','item_category_id']].drop_duplicates()
    
    all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
    
    del grid
    gc.collect()
       
    return all_data
    

Data clean up

In [ ]:
train_df = train_df[train_df.item_price < 100000]
train_df = train_df[train_df.item_cnt_day < 1001]

Check price 

In [ ]:
train_df[train_df["item_price"] <= 0]

In [ ]:
train_df[
    (train_df.shop_id==32)&
    (train_df.item_id==2973)&
    (train_df.date_block_num==4)&
    (train_df.item_price>0)]

In [ ]:
median = train_df[
    (train_df.shop_id==32)&
    (train_df.item_id==2973)&
    (train_df.date_block_num==4)&
    (train_df.item_price>0)].item_price.median()
print("median", median)

In [ ]:
train_df.loc[train_df["item_price"] <= 0, "item_price"] = median

In [ ]:
# Якутск Орджоникидзе, 56
train_df.loc[train_df.shop_id == 0, 'shop_id'] = 57
test_df.loc[test_df.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train_df.loc[train_df.shop_id == 1, 'shop_id'] = 58
test_df.loc[test_df.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train_df.loc[train_df.shop_id == 10, 'shop_id'] = 11
test_df.loc[test_df.shop_id == 10, 'shop_id'] = 11

In [ ]:
all_data = gen_feature_matrix(train_df, test_df, items_df, [1, 2, 3, 6, 12], index_cols)

In [ ]:
# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)
    

In [ ]:
all_data.head()

Price Trend Lag. Track the change of the latest price change of an item 


In [ ]:
train_df.head()

In [ ]:
group = train_df.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

all_data_price = pd.merge(all_data, group, on=['item_id'], how='left')
all_data_price['item_avg_item_price'] = all_data_price['item_avg_item_price'].astype(np.float16)

all_data_price.head()

In [ ]:
group = train_df.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

all_data_price = pd.merge(all_data_price, group, on=['date_block_num','item_id'], how='left')
all_data_price['date_item_avg_item_price'] = all_data_price['date_item_avg_item_price'].astype(np.float16)

all_data_price.head()

In [ ]:
lags = [1,2,3,4,5,6]
all_data_price = add_lag_features(all_data_price, lags, ['date_item_avg_item_price'], index_cols)
all_data_price.head()

In [ ]:
# find the price change of an item compare with the mean price along lagged months. Normalize the price change
for i in lags:
    all_data_price['delta_price_lag_'+str(i)] =  (all_data_price['date_item_avg_item_price_lag-'+ "{:02d}".format(i) ] - all_data_price['item_avg_item_price']) / all_data_price['item_avg_item_price']
    
all_data_price[all_data_price.date_block_num == 12].head()

In [ ]:
# select the nearest month with price change
def select_trend(row):
    for i in lags:
        if not row['delta_price_lag_'+str(i)] == 0:
            return row['delta_price_lag_'+str(i)]
    return 0

In [ ]:

all_data_price['delta_price_lag'] = all_data_price.apply(select_trend, axis=1)
all_data_price['delta_price_lag'] = all_data_price['delta_price_lag'].astype(np.float16)
all_data_price['delta_price_lag'].fillna(0, inplace=True)


fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag-{:02d}'.format(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]
                        
all_data_price.drop(fetures_to_drop, axis=1, inplace=True)

In [ ]:
all_data = all_data_price
all_data_price = None

In [ ]:
all_data.info()

### Mean encoding

In [ ]:
grouped = all_data.groupby(['date_block_num']).agg({'target': ['mean']})
grouped.columns = [ 'date_avg_target' ]
grouped.reset_index(inplace=True)


all_data = pd.merge(all_data, grouped, on=['date_block_num'], how='left')
all_data['date_avg_target'] = all_data['date_avg_target'].astype(np.float16)
all_data = add_lag_features(all_data, [1], ['date_avg_target'], index_cols)
all_data.drop(['date_avg_target'], axis=1, inplace=True)


In [ ]:

grouped = all_data.groupby(['date_block_num', 'item_id']).agg({'target': ['mean']})
grouped.columns = [ 'date_item_avg_target' ]
grouped.reset_index(inplace=True)

all_data = pd.merge(all_data, grouped, on=['date_block_num','item_id'], how='left')
all_data['date_item_avg_target'] = all_data['date_item_avg_target'].astype(np.float16)
all_data = add_lag_features(all_data, [1,2,3,6,12], ['date_item_avg_target'], index_cols)
all_data.drop(['date_item_avg_target'], axis=1, inplace=True)

In [ ]:
grouped = all_data.groupby(['date_block_num', 'shop_id']).agg({'target': ['mean']})
grouped.columns = [ 'date_shop_avg_target' ]
grouped.reset_index(inplace=True)

all_data = pd.merge(all_data, grouped, on=['date_block_num','shop_id'], how='left')
all_data['date_shop_avg_target'] = all_data['date_shop_avg_target'].astype(np.float16)
all_data = add_lag_features(all_data, [1,2,3,6,12], ['date_shop_avg_target'], index_cols)
all_data.drop(['date_shop_avg_target'], axis=1, inplace=True)

In [ ]:
grouped = all_data.groupby(['date_block_num', 'item_category_id']).agg({'target': ['mean']})
grouped.columns = [ 'date_item_category_avg_target' ]
grouped.reset_index(inplace=True)

all_data = pd.merge(all_data, grouped, on=['date_block_num','item_category_id'], how='left')
all_data['date_item_category_avg_target'] = all_data['date_item_category_avg_target'].astype(np.float16)
all_data = add_lag_features(all_data, [1], ['date_item_category_avg_target'], index_cols)
all_data.drop(['date_item_category_avg_target'], axis=1, inplace=True)

In [ ]:
grouped = all_data.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'target': ['mean']})
grouped.columns = [ 'date_shop_item_category_avg_target' ]
grouped.reset_index(inplace=True)

all_data = pd.merge(all_data, grouped, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
all_data['date_shop_item_category_avg_target'] = all_data['date_shop_item_category_avg_target'].astype(np.float16)
all_data = add_lag_features(all_data, [1], ['date_shop_item_category_avg_target'], index_cols)
all_data.drop(['date_shop_item_category_avg_target'], axis=1, inplace=True)

In [ ]:
all_data[all_data["date_block_num"] == 12].head()

The number of months since of the first sale of a shop-item pair and an item.

In [ ]:

first_month_shop_item = all_data.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
first_month_item = all_data.groupby(["item_id"])["date_block_num"].transform('min')

all_data['item_shop_first_sale'] = all_data['date_block_num'] - first_month_shop_item
all_data['item_first_sale'] = all_data['date_block_num'] - first_month_item


The month and number of days in that month

In [ ]:
all_data['month'] = all_data['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
all_data['days'] = all_data['month'].map(days).astype(np.int8)

Remove the first year samples as they have no lagged history data

In [ ]:
all_data = all_data[all_data["date_block_num"] >= 12]

Remove the target_shop and target_item as they are the info of current month

In [ ]:
all_data.drop(["target_shop", "target_item"], axis=1, inplace=True)

In [ ]:
all_data.head()

In [ ]:
all_data.info()

In [ ]:
print("all_data shape:", all_data.shape)
all_data.to_csv("all_data_lag.csv", index=False)

In [ ]:
shop_item_pairs = all_data[["shop_id", "item_id", "date_block_num"]] 
shop_item_pairs.to_csv("all_data_shop_item_pairs.csv", index=False)